In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
from IPython.display import display, HTML

In [ ]:
split_cost_csv_file="split-cost-allocation.csv"

# columns in the CUR csv file
account_id = "lineItem/UsageAccountId"
region = "product/region" 
product = "product/ProductName"
lineitem = "lineItem/Operation"
ec2_id = "splitLineItem/ParentResourceId"
split_cost = "splitLineItem/SplitCost"
split_unused_cost = "splitLineItem/UnusedCost"
ecs_service_name = "resourceTags/aws:ecs:serviceName"
ecs_task_id = "lineItem/ResourceId"
usage_hour = "lineItem/UsageStartDate"
usage_type = "lineItem/UsageType"
reservation = "splitLineItem/ReservedUsage"
actual = "splitLineItem/ActualUsage"
# usage type is CPU - USE2-ECS-EC2-vCPU-Hours or Memory - USE2-ECS-EC2-GB-Hours 

# values of interest, change these as needed 

account_id_val = XXX
region_val = "us-east-2"
product_val = "Amazon Elastic Container Service"
lineitem_val = "ECSTask-EC2"
usage_type_cpu = "vCPU-Hours"
usage_type_mem = "GB-Hours"

In [ ]:
# read csv and check its size as quick test for successful read
csv_df = pd.read_csv(split_cost_csv_file)
print(csv_df.shape)

In [ ]:
# filter for the ECS EC2 data, for specific region, and specific account


df = csv_df[(csv_df[product] == product_val)&\
            (csv_df[lineitem] == lineitem_val)&\
            (csv_df[region] == region_val)&\
            (csv_df[account_id] == account_id_val)]
# remove other columns that we don't care
df = df[[account_id, region, product, lineitem, ec2_id, split_cost, \
         split_unused_cost, ecs_service_name, ecs_task_id, usage_hour, usage_type, reservation, actual]]
print(df.shape)


In [ ]:
display(HTML(df.head(10).to_html()))

In [ ]:
service_names = df[ecs_service_name].dropna().unique()
usage_types = df[usage_type].dropna().unique()
ec2s = df[ec2_id].dropna().unique()
print(ec2s)
print(usage_types)
print(service_names)

In [ ]:
pivot = pd.pivot_table(df, values=[split_cost, split_unused_cost], index=usage_hour,
                       columns=ecs_service_name, aggfunc=np.sum)
plt.rcParams["figure.figsize"] = [20, 8]
ax = pivot.plot( kind='bar', stacked=True, title='Hourly Cost by Service')
# formatting
ax.set(xlabel='Usage Hour', ylabel='Hourly Cost')
ax.legend(title='ECS Service')
# removes the top and right lines from the figure to make it less boxy
for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)

In [ ]:
pivot_df = pd.pivot_table(df, values=[split_cost, split_unused_cost], index=ec2_id,
                       columns=ecs_service_name, aggfunc=np.sum)
display(HTML(pivot_df.to_html()))

In [ ]:
for svc_name in service_names:
    for ut in usage_types:
        plt_df = df[(df[ecs_service_name]==svc_name)&(df[usage_type]==ut)]
        plt.rcParams["figure.figsize"] = [20, 8]
        plt.rcParams["figure.autolayout"] = True
        plt.scatter(x=plt_df[usage_hour], y=plt_df[actual], label="Actual Usage", color="green")
        plt.scatter(x=plt_df[usage_hour], y=plt_df[reservation], label="Reserved Usage", color="blue", marker="_")
        plt.xticks(rotation=270)
        plt.legend(fontsize=16)
        plt.title(svc_name+"::"+ut, fontsize=16)
        plt.show()

In [ ]:
# get the sum of split cost and unused cost grouped by ecs service name
# you must have enabled ECS Managed Tag, set Propagate Tag to SERVICE, 
# and activated these ECS managed tags for use in cost report

#total_cost_by_service = df.groupby(ecs_service_name)[split_cost, split_unused_cost].sum()

pivot_df = pd.pivot_table(df, values=[split_cost, split_unused_cost], index=ecs_service_name,
                       columns=usage_type, aggfunc=np.sum)
display(HTML(pivot_df.to_html()))

In [ ]:

for svc_name in service_names:
    for ut in usage_types:
        plt_df = df[(df[ecs_service_name]==svc_name)&(df[usage_type]==ut)]
        plt.rcParams["figure.figsize"] = [20, 8]
        plt.rcParams["figure.autolayout"] = True
        plt.scatter(x=plt_df[usage_hour], y=plt_df[split_cost], label="Split Cost", color="green")
        plt.scatter(x=plt_df[usage_hour], y=plt_df[split_unused_cost], label="Split Unused Cost", color="blue", marker="_")
        plt.xticks(rotation=270)
        plt.legend()
        plt.title(svc_name+"::"+ut, fontsize=16)
        plt.show()
    